In [1]:
#импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm as tqdm
import urllib.request
import os

In [2]:
#у каждого студента свой вариант
VARIANT = 100
VARIANT_URL = f'https://storage.yandexcloud.net/plates/tasks/{VARIANT}.csv'

#создаем папку data и скачиваем файл с ссылками на изображения
os.makedirs('data', exist_ok = True)
urllib.request.urlretrieve(VARIANT_URL, f'data/{VARIANT}.csv');

df = pd.read_csv(f'data/{VARIANT}.csv')

#создаем папку images и скачиваем туда изображения
os.makedirs('data/images', exist_ok = True)
for url in tqdm(list(df['url'])[:]):
    name = url.split('/')[-1]
    path_img = f'data/images/{name}'
    if not os.path.exists(path_img):
        urllib.request.urlretrieve(url, path_img);
#если кто-то из студентов оптимизирует этот код
#и напишет параллельную загрузку изображений, пожалуйста, поделитесь им со своими однокурсниками

In [3]:
#устанавливаем open source проект nomeroff-net для распознавания номеров
!pip3 install 'git+https://github.com/facebookresearch/detectron2.git'
!git clone https://github.com/ria-com/nomeroff-net.git
!cd nomeroff-net ; git clone https://github.com/youngwanLEE/centermask2.git
!pip3 install 'git+https://github.com/ria-com/nomeroff-net.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-x1eh979u
  Created wheel for detectron2: filename=detectron2-0.3-cp36-cp36m-linux_x86_64.whl size=4987699 sha256=9578ae0d81ebfc92eec7cf251e8306dbf79f0dcd57b83de990c90a2d979783e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-cjssq9fb/wheels/2f/3b/c1/00424ff1494f5bd9f617a9b97453ad806595044a9cef3081da
Successfully built detectron2


  Attempting uninstall: detectron2
    Found existing installation: detectron2 0.2.1
    Uninstalling detectron2-0.2.1:
      Successfully uninstalled detectron2-0.2.1
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Cloning into 'nomeroff-net'...
remote: Enumerating objects: 398, done.
remote: Counting objects: 100% (398/398), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 3124 (delta 216), reused 267 (delta 113), pack-reused 2726
Receiving objects: 100% (3124/3124), 119.35 MiB | 2.04 MiB/s, done.
Resolving deltas: 100% (1821/1821), done.
Checking out files: 100% (172/172), done.
Cloning into 'centermask2'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 174 (delta 79), reused 140 (delta 56), pack-reused 0
Receiving objects: 100% (174/174), 88.45 KiB | 363.00 KiB/s, done.
Resolving deltas: 100% (79/79), done.
 

  Created wheel for nomeroff-net: filename=nomeroff_net-1.0.0-py3-none-any.whl size=37485 sha256=5f69b21c19dbdc81a297140a65241828b8565defa6d5108282ab098874df0549
  Stored in directory: /tmp/pip-ephem-wheel-cache-3iyezvln/wheels/35/37/81/f5b392e474f6410f072164e0c96f46086a0c7645573e4bcba9
Successfully built nomeroff-net
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
#нейронку можно запустить на GPU, либо на CPU
import os
import pickle
#os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference

# dynamically grow the memory used on the GPU
from tensorflow.compat.v1.keras.backend import set_session
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True 
sess = tf.compat.v1.Session(config=config)
set_session(sess)

# Import all necessary libraries.
import numpy as np
import sys
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#В https://github.com/ria-com/nomeroff-net/issues/88 рассказывается как получить confidence
#Но функция get_acc устарела и при tensorflow>=2.3.* не работает, тут приведена исправленная функция
def get_acc_(predicted, decode,region):
    labels = []
    detector = textDetector.detectors[int(textDetector.detectors_map[region])]
    for text in decode:
        labels.append(detector.text_to_labels(text))
    loss = tf.keras.backend.ctc_batch_cost(
        np.array(labels), np.array(predicted)[:, 2:, :], 
        np.array([[detector.label_length] for label in labels]), 
        np.array([[detector.max_text_len] for label in labels])
    )
    return  1 - loss

def get_acc(predicted, decode, regions):
    acc = []
    for i, region in enumerate(regions):
        if textDetector.detectors_map.get(region, None) is None or len(decode[i]) == 0:
            acc.append([0])
        else:
            detector = textDetector.detectors[int(textDetector.detectors_map[region])]
            _acc = get_acc_([predicted[i]], [decode[i]],region)
            acc.append(_acc[0])
    return acc

In [4]:
import os
import numpy as np
import sys
import matplotlib.image as mpimg

# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('nomeroff-net/')
sys.path.append(NOMEROFF_NET_DIR)

# Import license plate recognition tools.
from NomeroffNet import Detector
from NomeroffNet import filters
from NomeroffNet import RectDetector
from NomeroffNet import OptionsDetector
from NomeroffNet import TextDetector
from NomeroffNet import textPostprocessing

# load models
rectDetector = RectDetector()

optionsDetector = OptionsDetector()
optionsDetector.load("latest")

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": "latest"
    },
    "eu_ua_1995": {
        "for_regions": ["eu_ua_1995"],
        "model_path": "latest"
    },
    "eu": {
        "for_regions": ["eu"],
        "model_path": "latest"
    },
    "ru": {
        "for_regions": ["ru", "eu-ua-fake-lnr", "eu-ua-fake-dnr"],
        "model_path": "latest" 
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": "latest"
    },
    "ge": {
        "for_regions": ["ge"],
        "model_path": "latest"
    }
})


nnet = Detector()
nnet.loadModel(NOMEROFF_NET_DIR)

Config '/var/www/nomeroff-net/romankucev/Desktop/shad/solution/nomeroff-net/./NomeroffNet/configs/centermask2/numberplates/cpu_centermask_numberplate_V_39_eSE_FPN_ms_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [5]:
if 'OCR' not in os.listdir():
    os.mkdir('OCR')
if 'cv_img_mask_npz' not in os.listdir():
    os.mkdir('cv_img_mask_npz')
images = list(filter(lambda x: x[0] !='.',os.listdir('data/images')))
#теперь прогоним все наши изображения из папки images через нейронку
for img_name in tqdm(images):
    try:
        #нейронка выдает довольно много информации, будем ее записывать в отдельный файл для каждого изображения
        d = {}
        d['img_name'] = img_name
        img_path = 'data/images/'+img_name
        img = mpimg.imread(img_path)
        cv_img_mask = nnet.detect_mask([img])
        #не знаю почему, но иногда нейронка выдает несколько масок для одного изображения
        #в этих случаях считаю, что нейронка сломалась и не нашла ни одного номера
        if len(cv_img_mask) != 1:
            print(img_name,'--',len(cv_img_mask))
            continue
        #запишем координаты найденных номеров и другие параметры
        arrPoints = rectDetector.detect(cv_img_mask[0])
        d['arrPoints'] = arrPoints
        zones = rectDetector.get_cv_zonesBGR(img, arrPoints)
        d['zones'] = zones
        regionIds, stateIds, countLines = optionsDetector.predict(zones)
        d['regionIds'] = regionIds
        d['stateIds'] = stateIds
        d['countLines'] = countLines
        regionNames = optionsDetector.getRegionLabels(regionIds)
        d['regionNames'] = regionNames
        textArr, b = textDetector.predict(zones, regionNames, countLines, return_acc=True);
        textArr = textPostprocessing(textArr, regionNames)
        #в textArr запишем распознанные номера
        d['textArr'] = textArr
        acc = np.array(get_acc(b, textArr, regionNames))
        #в acc запишем confidence 
        d['acc'] = acc
        #в папку OCR запишем наш словарь d
        with open('OCR/' + img_name.split('.')[0] + '.pickle', 'wb') as f:
            pickle.dump(d, f)
        #в cv_img_mask_npz запишем макси с найденными номерами
        np.savez_compressed('cv_img_mask_npz/'+ img_name.split('.')[0] + '.npz', cv_img_mask)
    except:
        pass

/var/www/nomeroff-net/romankucev/Desktop/shad/solution/nomeroff-net/centermask2/centermask/modeling/fcos/fcos_outputs.py:402: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  per_candidate_nonzeros = per_candidate_inds.nonzero()
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [6]:
import pandas as pd
import pickle
import tqdm
import numpy as np
from tqdm import tqdm_notebook

#преобразуем номер к стандартному виду
def rename_plate(s):
    dictionary = {" ":"", "y":"у",'a':'а','b':'в','c':'с','e':'е','h':'н','k':'к','m':'м','o':'о','p':'р',
                 't':'т','x':'х','д':'d'} 
    s = s.lower()
    for key in dictionary.keys():
        s = s.replace(key, dictionary[key])
    return s.upper()

#напишем функцию, которая определеяет площадь полигона 
def PolygonArea(corners):
    n = len(corners) # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

In [10]:
#у нас может быть несколько номеров на изображении
#в качестве ответа нам нужно указать номер главного автомобиля, который ближе всего расположен к камере
#обычно у этого автомобиля номер имеет самую большую площадь на изображении
#поэтому логика такая: отбираем все номера с confidence>0.5
#среди всех полученных номеров выбираем самый большой номер
for i in tqdm_notebook(df.index[:]):
    img_name = df.loc[i,'id']
    try:
        with open('OCR/' + img_name.split('.')[0]+'.pickle', 'rb') as f:
            ocr_dict = pickle.load(f)
            arrPoints  = ocr_dict['arrPoints']
            area = np.array(list(map(PolygonArea,arrPoints)))
            acc = ocr_dict['acc'].reshape((1, -1))[0]
            mask_threshold = acc > 0.5
            filtred_acc = acc[mask_threshold]
            filtred_area = area[mask_threshold]
            filtered_plates = np.array(ocr_dict['textArr'])[mask_threshold]
            if len(filtered_plates) == 0:
                df.loc[i,'predict'] = ''
            else:
                plate = filtered_plates[filtred_area.argmax()]
                plate = rename_plate(plate)
                accuracy = filtred_acc[filtred_area.argmax()]
                df.loc[i,'predict'] = plate
                df.loc[i,'accuracy'] = accuracy
    except:
        df.loc[i,'predict'] = ''

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [13]:
df.to_csv('data/100_with_prediction_nn.csv',index=None)